In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import DataFrameLoader
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

# Load environment variables from a .env file
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
import pandas as pd

file_path = ('./files/jeju/성별카드이용금액비율.csv') # insert the path of the csv file
df = pd.read_csv(file_path, on_bad_lines="skip")

# 임베딩 모델 설정
embeddings = OpenAIEmbeddings()

# 텍스트 데이터와 메타데이터 추출
texts = df["sex"].tolist()
metadatas = df.drop(columns=["sex"]).to_dict(orient="records")  # 나머지 열은 메타데이터로 처리

# FAISS Vectorstore 생성
vectorstore = FAISS.from_texts(texts, embeddings, metadatas=metadatas)

# Vectorstore 저장
faiss_file_path = "index_1.faiss"
vectorstore.save_local(faiss_file_path)
print(f"FAISS Vectorstore 저장됨: {faiss_file_path}")

FAISS Vectorstore 저장됨: index_1.faiss


In [ ]:
# FAISS Vectorstore 로드
vectorstore = FAISS.load_local(faiss_file_path, embeddings, allow_dangerous_deserialization=True)

# FAISS에서 데이터 추출
documents = vectorstore.docstore._dict.values()

# 텍스트와 메타데이터 복구
data = []
for doc in documents:
    row = {
        "sex": doc.page_content,  # 텍스트 데이터
        **doc.metadata                # 메타데이터를 펼쳐서 추가
    }
    data.append(row)

# 복구된 데이터를 데이터프레임으로 변환
recovered_df = pd.DataFrame(data)



# CSV로 저장
output_csv_file = "recovered.csv"
recovered_df.to_csv(output_csv_file, index=False, encoding="utf-8")
print(f"CSV 파일로 복구 완료: {output_csv_file}")

CSV 파일로 복구 완료: recovered.csv


In [8]:
from langchain_community.document_loaders import DataFrameLoader
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

base_dir = './files/jeju'

embeddings = OpenAIEmbeddings()

faiss_file_paths = []
for roots, dirs, files in os.walk(base_dir):
    for i, file in enumerate(files):
        file_path = f'{base_dir}/{file}'
        df = pd.read_csv(file_path, on_bad_lines="skip")
        df = df.drop(df[df['age_range'] == '알수없음'].index)
        
        if '성별' in file:
            # 텍스트 데이터와 메타데이터 추출
            texts = df["sex"].tolist()
            metadatas = df.drop(columns=["sex"]).to_dict(orient="records")
            
            # FAISS Vectorstore 생성
            vectorstore = FAISS.from_texts(texts, embeddings, metadatas=metadatas)
            
            # Vectorstore 저장
            faiss_file_path = f"index_{i+1}.faiss"
            faiss_file_paths.append(faiss_file_path)
            vectorstore.save_local(faiss_file_path)
            print(f"FAISS Vectorstore 저장됨: {faiss_file_path}")

        elif '시간' in file:
            # 텍스트 데이터와 메타데이터 추출
            texts = df["time_type"].tolist()
            metadatas = df.drop(columns=["time_type"]).to_dict(orient="records")
            
            # FAISS Vectorstore 생성
            vectorstore = FAISS.from_texts(texts, embeddings, metadatas=metadatas)
            
            # Vectorstore 저장
            faiss_file_path = f"index_{i+1}.faiss"
            faiss_file_paths.append(faiss_file_path)
            vectorstore.save_local(faiss_file_path)
            print(f"FAISS Vectorstore 저장됨: {faiss_file_path}")

        elif '요일' in file:
            # 텍스트 데이터와 메타데이터 추출
            texts = df["day_of_week"].tolist()
            metadatas = df.drop(columns=["day_of_week"]).to_dict(orient="records")
            
            # FAISS Vectorstore 생성
            vectorstore = FAISS.from_texts(texts, embeddings, metadatas=metadatas)
            
            # Vectorstore 저장
            faiss_file_path = f"index_{i+1}.faiss"
            faiss_file_paths.append(faiss_file_path)
            vectorstore.save_local(faiss_file_path)
            print(f"FAISS Vectorstore 저장됨: {faiss_file_path}")

        else:
            print(f'Cannot find a {file} file.')

FAISS Vectorstore 저장됨: index_1.faiss
FAISS Vectorstore 저장됨: index_2.faiss
FAISS Vectorstore 저장됨: index_3.faiss


In [ ]:
from langchain.tools import Tool

tools = []

for i, faiss_file_path in enumerate(faiss_file_path):
    # FAISS Vectorstore 로드
    vectorstore = FAISS.load_local(faiss_file_path, embeddings, allow_dangerous_deserialization=True)

    # FAISS에서 데이터 추출
    documents = vectorstore.docstore._dict.values()

    # 텍스트와 메타데이터 복구
    data = []
    if i == 0:
        for doc in documents:
            row = {
                "sex": doc.page_content,  # 텍스트 데이터
                **doc.metadata                # 메타데이터를 펼쳐서 추가
            }
            data.append(row)
            
        # 복구된 데이터를 데이터프레임으로 변환
        df_gender = pd.DataFrame(data)
        
        for name, df in df_gender.items():
            # 각각의 CSV 파일을 하나의 Tool로 등록
            tool = Tool(
                name=name,
                func=lambda query, df=df: df.query(query),
                description=f" {name} 데이터셋을 분석하고, 결과를 한국어로 답변하세요."
            )
            tools.append(tool)
        
    elif i == 1:
        for doc in documents:
            row = {
                "time_type": doc.page_content,  # 텍스트 데이터
                **doc.metadata                # 메타데이터를 펼쳐서 추가
            }
            data.append(row)

        # 복구된 데이터를 데이터프레임으로 변환
        df_time = pd.DataFrame(data)

    elif i == 2:
        for doc in documents:
            row = {
                "day_of_week": doc.page_content,  # 텍스트 데이터
                **doc.metadata                # 메타데이터를 펼쳐서 추가
            }
            data.append(row)

        # 복구된 데이터를 데이터프레임으로 변환
        df_day = pd.DataFrame(data)